In [5]:
# wget https://raw.githubusercontent.com/Vaseekaran-V/Datastorm-3/main/data/train_data_modified.csv?token=GHSAT0AAAAAABS4N64JYNY6EXMSZROSSTSOYSJQHMQ
# !wget https://raw.githubusercontent.com/Vaseekaran-V/Datastorm-3/main/data/validation_data.csv?token=GHSAT0AAAAAABS4N64JIDICIDNZNGXSE2HGYSIQRIQ
# !wget https://raw.githubusercontent.com/Vaseekaran-V/Datastorm-3/main/data/test_data.csv?token=GHSAT0AAAAAABS4N64J3BGH5UCQAEMW3NK2YSIQR7A

--2022-03-27 11:15:54--  https://raw.githubusercontent.com/Vaseekaran-V/Datastorm-3/main/data/train_data_modified.csv?token=GHSAT0AAAAAABS4N64JYNY6EXMSZROSSTSOYSJQHMQ
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100306 (98K) [text/plain]
Saving to: ‘train_data_modified.csv?token=GHSAT0AAAAAABS4N64JYNY6EXMSZROSSTSOYSJQHMQ’

train_data_modified 100%[===================>]  97.96K  --.-KB/s    in 0.01s   

2022-03-27 11:15:54 (6.67 MB/s) - ‘train_data_modified.csv?token=GHSAT0AAAAAABS4N64JYNY6EXMSZROSSTSOYSJQHMQ’ saved [100306/100306]



## Import Libraries

In [14]:
def week_id_valid(week):
  if week=="w1":    return 1
  elif week=="w2":    return 2
  elif week=="w3": return 3
  else: return 4

In [167]:
from sklearn.metrics import mean_absolute_error , mean_absolute_percentage_error
def calculate_results(y_true, y_pred):
  return {'mae':mean_absolute_error(y_true, y_pred),
          'mape': mean_absolute_percentage_error(y_true, y_pred)}

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## https://www.mariofilho.com/how-to-predict-multiple-time-series-with-scikit-learn-with-sales-forecasting-example/

In [86]:
train_data = pd.read_csv("/content/train_data_modified.csv?token=GHSAT0AAAAAABS4N64JYNY6EXMSZROSSTSOYSJQHMQ")
train_data

,CategoryCode,ItemCode,Week,id,WeeklySales
0,category_2,174436,w1,1,35
1,category_2,118033,w1,1,7
2,category_2,20824,w1,1,53
3,category_1,1061341,w1,1,18
4,category_1,76399,w1,1,21
...,...,...,...,...,...
3767,category_1,1044619,w4,20,45
3768,category_4,23200,w4,20,22
3769,category_1,1084498,w4,20,7
3770,category_4,872260,w4,20,12


In [87]:
valid_data = pd.read_csv('/content/validation_data.csv?token=GHSAT0AAAAAABS4N64IO67OYT625ZOBIP5QYSHJT2Q')
valid_data['id'] = [week_id_valid(week)+20 for week in valid_data['Week']]
valid_data.sort_values(by='id', inplace = True)
valid_data

,CategoryCode,ItemCode,Week,WeeklySales,id
0,category_2,1044502,w1,11,21
142,category_1,48940,w1,22,21
269,category_1,35584,w1,15,21
138,category_2,59749,w1,33,21
137,category_2,132028,w1,112,21
...,...,...,...,...,...
139,category_2,118033,w4,12,24
140,category_2,1098502,w4,4,24
268,category_2,1010068,w4,19,24
146,category_2,211309,w4,23,24


In [20]:
test_data = pd.read_csv('/content/test_data.csv?token=GHSAT0AAAAAABS4N64J3BGH5UCQAEMW3NK2YSIQR7A')
test_data['id'] = [week_id_valid(week)+20 for week in test_data['Week']]
test_data

,CategoryCode,ItemCode,Week,PredictedSales,id
0,category_1,43738,w4,NaN,24
1,category_2,1006090,w1,NaN,21
2,category_2,1076929,w4,NaN,24
3,category_1,1081321,w3,NaN,23
4,category_2,216151,w4,NaN,24
...,...,...,...,...,...
372,category_2,1101571,w1,NaN,21
373,category_2,1090258,w4,NaN,24
374,category_2,906595,w1,NaN,21
375,category_2,32245,w1,NaN,21


## Adding train and valid dataset together

In [29]:
train_valid = pd.concat([train_data, valid_data])
train_valid

,CategoryCode,ItemCode,Week,id,WeeklySales
0,category_2,174436,w1,1,35
1,category_2,118033,w1,1,7
2,category_2,20824,w1,1,53
3,category_1,1061341,w1,1,18
4,category_1,76399,w1,1,21
...,...,...,...,...,...
139,category_2,118033,w4,24,12
140,category_2,1098502,w4,24,4
268,category_2,1010068,w4,24,19
146,category_2,211309,w4,24,23


In [25]:
train_valid.drop(["Week"], axis=1, inplace = True)

## Feature  Engineering

In [139]:
df = train_valid.copy()
train_test = train_data.copy()
test_df = test_data.copy()

test_df.columns = ['CategoryCode','ItemCode','Week','WeeklySales','id']
test_df


# from sklearn.preprocessing import LabelEncoder
# le= LabelEncoder()
# df['Week'] = le.fit_transform(df['Week'].values)
# train_test['Week'] = le.fit_transform(train_test['Week'].values)
# test_df['Week'] = le.transform(test_df['Week'].values)


# df.drop(['CategoryCode'], axis=1, inplace = True)
# train_test.drop(['CategoryCode'], axis = 1, inplace = True)
# test_df.drop(['CategoryCode'], axis = 1 , inplace = True)

,CategoryCode,ItemCode,Week,WeeklySales,id
0,category_1,43738,w4,NaN,24
1,category_2,1006090,w1,NaN,21
2,category_2,1076929,w4,NaN,24
3,category_1,1081321,w3,NaN,23
4,category_2,216151,w4,NaN,24
...,...,...,...,...,...
372,category_2,1101571,w1,NaN,21
373,category_2,1090258,w4,NaN,24
374,category_2,906595,w1,NaN,21
375,category_2,32245,w1,NaN,21


In [140]:
df['Last_Week_Sales'] = df.groupby(['ItemCode'])['WeeklySales'].shift()
df['Last_Week_Diff'] = df.groupby(['ItemCode'])['Last_Week_Sales'].shift(periods=1) - df.groupby(['ItemCode'])['Last_Week_Sales'].shift(periods=2)
df['Last-1_Week_Sales'] = df.groupby(['ItemCode'])['WeeklySales'].shift(2)
df['Last-1_Week_Diff'] = df.groupby(['ItemCode'])['Last-1_Week_Sales'].shift(periods=2) - df.groupby(['ItemCode'])['Last_Week_Sales'].shift(periods=3)
df['Last-2_Week_Sales'] = df.groupby(['ItemCode'])['WeeklySales'].shift(3)
df['Last-2_Week_Diff'] = df.groupby(['ItemCode'])['Last-2_Week_Sales'].shift(periods=1) - df.groupby(['ItemCode'])['Last_Week_Sales'].shift(periods=3)
df = df.dropna()
df.head()

,CategoryCode,ItemCode,Week,id,WeeklySales,Last_Week_Sales,Last_Week_Diff,Last-1_Week_Sales,Last-1_Week_Diff,Last-2_Week_Sales,Last-2_Week_Diff
738,category_1,3418,w1,5,44,41.0,-1.0,41.0,0.0,42.0,0.0
739,category_1,1050046,w1,5,22,16.0,3.0,12.0,0.0,9.0,0.0
740,category_2,1092184,w1,5,25,12.0,1.0,16.0,0.0,15.0,0.0
741,category_3,1064572,w1,5,14,7.0,-1.0,6.0,0.0,7.0,0.0
742,category_2,1067074,w1,5,29,39.0,-11.0,27.0,0.0,38.0,0.0


In [117]:
train_test_copy = train_test.copy()

In [54]:
from lightgbm import LGBMRegressor

In [164]:
def rmsle(ytrue, ypred):
    return np.sqrt(mean_squared_log_error(ytrue, ypred))

In [170]:
c  = []
mean_error = []
for week in range(21,25):

    train = df[df['id'] < week]
    val = df[df['id'] == week]

    xtr, xts = train.drop(['WeeklySales', 'CategoryCode','Week'], axis=1), val.drop(['WeeklySales', 'CategoryCode','Week'], axis=1)
    ytr, yts = train['WeeklySales'].values, val['WeeklySales'].values

    mdl = LGBMRegressor(n_estimators=10000, learning_rate=0.01)
    mdl.fit(xtr, np.log1p(ytr))

    p = np.expm1(mdl.predict(xts)) # on validation

    error = calculate_results(yts, p)
    print(f'Week {week} - Error {error}')
    mean_error.append(error)

    train_test = pd.concat([train_test, test_df[test_df['id']==week]])
    # train_test['Average_Sales] = train_test.groupby(['ItemCode'])['WeeklySales'].mean()
    train_test['Last_Week_Sales'] = train_test.groupby(['ItemCode'])['WeeklySales'].shift()
    train_test['Last_Week_Diff'] = train_test.groupby(['ItemCode'])['Last_Week_Sales'].shift(periods=1) - train_test.groupby(['ItemCode'])['Last_Week_Sales'].shift(periods=2)
    train_test['Last-1_Week_Sales'] = train_test.groupby(['ItemCode'])['WeeklySales'].shift(2)
    train_test['Last-1_Week_Diff'] = train_test.groupby(['ItemCode'])['Last-1_Week_Sales'].shift(periods=2) - train_test.groupby(['ItemCode'])['Last_Week_Sales'].shift(periods=3)
    train_test['Last-2_Week_Sales'] = train_test.groupby(['ItemCode'])['WeeklySales'].shift(3)
    train_test['Last-2_Week_Diff'] = train_test.groupby(['ItemCode'])['Last-2_Week_Sales'].shift(periods=1) - train_test.groupby(['ItemCode'])['Last_Week_Sales'].shift(periods=3)
    # train_test = df.dropna()

    train = train_test[train_test['id'] < week]
    train = train.dropna()
    test = train_test[train_test['id']==week]

    xtest = test.drop(['WeeklySales', 'CategoryCode','Week'], axis=1) 
    ytest = test['WeeklySales'].values

    test_p = np.expm1(mdl.predict(xtest)) # test data
    # test_df[test_df['id']==week]['WeeklySales'] = test_p

    c.append({'CategoryCode': test['CategoryCode'],'ItemCode':test['ItemCode'], 'Week':test['Week'],'id':test['id'] ,'WeeklySales':test_p})

# print('Mean Error = %.5f' % np.mean(mean_error))

Week 21 - Error {'mae': 11.488699713659022, 'mape': 0.6094027269423337}
Week 22 - Error {'mae': 18.869706773420223, 'mape': 0.6114119865800174}
Week 23 - Error {'mae': 22.87174650673536, 'mape': 1.08900380865578}
Week 24 - Error {'mae': 18.7418136907078, 'mape': 0.5969743073611311}


In [171]:
mape = []
for error in mean_error:
  mape.append(error['mape'])
mape

[0.6094027269423337, 0.6114119865800174, 1.08900380865578, 0.5969743073611311]

In [142]:
pd.DataFrame(c[1])

,CategoryCode,ItemCode,Week,id,WeeklySales
10,category_2,1090258,w2,22,14.497366
11,category_1,1032559,w2,22,15.357595
14,category_1,1067119,w2,22,14.259390
15,category_1,1056463,w2,22,13.885248
16,category_2,1101571,w2,22,8.705112
...,...,...,...,...,...
358,category_2,262384,w2,22,17.464980
359,category_1,38518,w2,22,14.432468
361,category_2,111436,w2,22,11.805257
371,category_2,32245,w2,22,20.308279


In [145]:
submissions = test_data.drop(['PredictedSales'], axis = 1)
submissions

,CategoryCode,ItemCode,Week,id
0,category_1,43738,w4,24
1,category_2,1006090,w1,21
2,category_2,1076929,w4,24
3,category_1,1081321,w3,23
4,category_2,216151,w4,24
...,...,...,...,...
372,category_2,1101571,w1,21
373,category_2,1090258,w4,24
374,category_2,906595,w1,21
375,category_2,32245,w1,21


In [152]:
all_concat = pd.concat([pd.DataFrame(c[0]), pd.DataFrame(c[1]),pd.DataFrame(c[2]),pd.DataFrame(c[3])])
all_concat

,CategoryCode,ItemCode,Week,id,WeeklySales
1,category_2,1006090,w1,21,12.783702
7,category_4,23200,w1,21,80.415427
8,category_1,1068883,w1,21,4.940167
12,category_4,123307,w1,21,176.127370
13,category_2,837943,w1,21,117.118860
...,...,...,...,...,...
357,category_1,1044610,w4,24,9.347927
364,category_2,87559,w4,24,5.958055
365,category_2,57058,w4,24,5.987910
369,category_1,1049776,w4,24,11.293492


In [153]:
submissions1 = pd.merge(submissions,all_concat, on=["CategoryCode","ItemCode","Week","id"], how = "left")
submissions1

,CategoryCode,ItemCode,Week,id,WeeklySales
0,category_1,43738,w4,24,8.425899
1,category_2,1006090,w1,21,12.783702
2,category_2,1076929,w4,24,8.122628
3,category_1,1081321,w3,23,10.551371
4,category_2,216151,w4,24,5.928541
...,...,...,...,...,...
372,category_2,1101571,w1,21,9.276697
373,category_2,1090258,w4,24,14.287120
374,category_2,906595,w1,21,9.173033
375,category_2,32245,w1,21,53.779427


In [154]:
submissions1['ID'] = [str(category)+'_'+str(item)+'_'+str(week) for category,item,week in zip(submissions1['CategoryCode'], submissions1['ItemCode'],submissions1['Week'])]
submissions1['ID']

0        category_1_43738_w4
1      category_2_1006090_w1
2      category_2_1076929_w4
3      category_1_1081321_w3
4       category_2_216151_w4
               ...          
372    category_2_1101571_w1
373    category_2_1090258_w4
374     category_2_906595_w1
375      category_2_32245_w1
376    category_2_1006090_w2
Name: ID, Length: 377, dtype: object

In [155]:
submissions1[['ID','WeeklySales']].to_csv("submissions_3.csv", index = False)

In [159]:
submissions0 = pd.read_csv('/content/submission_0_naive_forecast_with_average (7).csv')
submissions0

,ID,WeeklySales
0,category_2_87559_w1,24.300000
1,category_2_9925_w1,11.900000
2,category_2_999403_w1,15.150000
3,category_2_1103056_w1,16.333333
4,category_2_838456_w1,15.850000
...,...,...
372,category_2_87559_w4,24.300000
373,category_3_37861_w4,135.450000
374,category_1_1067119_w4,25.400000
375,category_1_839374_w4,40.200000


In [163]:
pd.merge(submissions0[['ID','WeeklySales']], submissions1[['ID','WeeklySales']] , on = ['ID'])

,ID,WeeklySales_x,WeeklySales_y
0,category_2_87559_w1,24.300000,31.477262
1,category_2_9925_w1,11.900000,17.721454
2,category_2_999403_w1,15.150000,16.840274
3,category_2_1103056_w1,16.333333,11.284036
4,category_2_838456_w1,15.850000,11.421636
...,...,...,...
372,category_2_87559_w4,24.300000,5.958055
373,category_3_37861_w4,135.450000,7.448526
374,category_1_1067119_w4,25.400000,9.539693
375,category_1_839374_w4,40.200000,7.225629
